In [1]:
import pymysql
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
prod_conn = pymysql.connect(
  host='localhost',
  user='root',
  password='Mike4568!',
  port=3306,
  database='FORMULA1')
with prod_conn:
    cur = prod_conn.cursor()

In [282]:
cur.execute("SHOW DATABASES;")

4

In [8]:
print(cur.fetchall())

(('FORMULA1',), ('information_schema',), ('mysql',), ('performance_schema',))


In [9]:
cur.execute("SHOW TABLES;")

12

In [10]:
print(cur.fetchall())

(('circuits',), ('constructor_results',), ('constructor_standings',), ('constructors',), ('driver_standings',), ('drivers',), ('lap_times',), ('pit_stops',), ('qualifying',), ('races',), ('results',), ('status',))


In [304]:
test1 = """
            SELECT lt.raceId, lt.driverId, lt.lap AS lt_lap, lt.position, lt.milliseconds, ps.lap AS ps_lap
            FROM lap_times lt
            INNER JOIN races rs on lt.raceId = rs.raceId 
            LEFT JOIN pit_stops ps ON lt.raceId=ps.raceId
            WHERE (lt.raceId = 944) AND (ps.raceId IS NULL)
            LIMIT 100;
"""

In [7]:
another_one = """
SELECT *
FROM (
    SELECT lt.raceId, lt.driverId, lt.lap AS lt_lap, lt.position, lt.milliseconds, lt.lap, rs.circuitId, rs.date 
    FROM lap_times lt
    INNER JOIN races rs on lt.raceId = rs.raceId
) a

LEFT JOIN

(
   SELECT ps.raceId AS rd, ps.lap AS ps_lap, ps.driverId AS di, ps.millisec as pit_time
    FROM pit_stops ps
) b

ON (a.raceId = b.rd) AND (a.driverid = b.di) AND (a.lap = b.ps_lap);
"""

In [8]:
res1 = data_retr(another_one)

In [9]:
res1.head()

,raceId,driverId,lt_lap,position,milliseconds,lap,circuitId,date,rd,ps_lap,di,pit_time
0,841,153,1,21,133879,1,1,2011-03-27,841.0,1.0,153.0,26898.0
1,841,30,1,22,161428,1,1,2011-03-27,841.0,1.0,30.0,25021.0
2,841,17,11,7,114995,11,1,2011-03-27,841.0,11.0,17.0,23426.0
3,841,4,12,12,112624,12,1,2011-03-27,841.0,12.0,4.0,23251.0
4,841,13,13,11,114060,13,1,2011-03-27,841.0,13.0,13.0,23842.0


In [10]:
final = res1.drop(columns=['rd', 'di','lap'])

In [11]:
final.head()

,raceId,driverId,lt_lap,position,milliseconds,circuitId,date,ps_lap,pit_time
0,841,153,1,21,133879,1,2011-03-27,1.0,26898.0
1,841,30,1,22,161428,1,2011-03-27,1.0,25021.0
2,841,17,11,7,114995,1,2011-03-27,11.0,23426.0
3,841,4,12,12,112624,1,2011-03-27,12.0,23251.0
4,841,13,13,11,114060,1,2011-03-27,13.0,23842.0


In [466]:
final.isnull().sum()

raceId               0
driverId             0
lt_lap               0
position             0
milliseconds         0
circuitId            0
date                 0
ps_lap          465068
dtype: int64

In [471]:
final.loc[final['ps_lap'].isnull()]

,raceId,driverId,lt_lap,position,milliseconds,circuitId,date,ps_lap
360,841,20,1,1,98109,1,2011-03-27,NaN
361,841,20,2,1,93006,1,2011-03-27,NaN
362,841,20,3,1,92713,1,2011-03-27,NaN
363,841,20,4,1,92803,1,2011-03-27,NaN
364,841,20,5,1,92342,1,2011-03-27,NaN
...,...,...,...,...,...,...,...,...
472499,1030,842,49,18,103719,24,2019-12-01,NaN
472500,1030,842,50,18,102820,24,2019-12-01,NaN
472501,1030,842,51,18,102814,24,2019-12-01,NaN
472502,1030,842,52,18,105254,24,2019-12-01,NaN


In [491]:
final['ps_lap'].values[0]

1.0

In [495]:
pd.notna(final['ps_lap'].values[0])

True

In [12]:
final['target'] = final['ps_lap'].map(pd.notna)
final.head()

,raceId,driverId,lt_lap,position,milliseconds,circuitId,date,ps_lap,pit_time,target
0,841,153,1,21,133879,1,2011-03-27,1.0,26898.0,True
1,841,30,1,22,161428,1,2011-03-27,1.0,25021.0,True
2,841,17,11,7,114995,1,2011-03-27,11.0,23426.0,True
3,841,4,12,12,112624,1,2011-03-27,12.0,23251.0,True
4,841,13,13,11,114060,1,2011-03-27,13.0,23842.0,True


In [516]:
final.head()

,raceId,driverId,lt_lap,position,milliseconds,circuitId,date,ps_lap,target
0,841,153,1,21,133879,1,2011,1.0,True
1,841,30,1,22,161428,1,2011,1.0,True
2,841,17,11,7,114995,1,2011,11.0,True
3,841,4,12,12,112624,1,2011,12.0,True
4,841,13,13,11,114060,1,2011,13.0,True


In [13]:
final.to_csv("./data/Final_2.csv")

In [509]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472504 entries, 0 to 472503
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   raceId        472504 non-null  int64  
 1   driverId      472504 non-null  int64  
 2   lt_lap        472504 non-null  int64  
 3   position      472504 non-null  int64  
 4   milliseconds  472504 non-null  int64  
 5   circuitId     472504 non-null  int64  
 6   date          472504 non-null  object 
 7   ps_lap        7436 non-null    float64
 8   target        472504 non-null  bool   
dtypes: bool(1), float64(1), int64(6), object(1)
memory usage: 29.3+ MB


In [513]:
final.date = [x.year for x in final.date]

In [442]:
final.columns[7]

'raceId'

In [332]:
res = data_retr(another_one)

In [391]:
joined = """ 
    SELECT lt.raceId, lt.driverId, lt.lap AS lt_lap, lt.position, lt.milliseconds, lt.lap, rs.circuitId, rs.date 
    FROM lap_times lt
    INNER JOIN races rs on lt.raceId = rs.raceId

"""

In [307]:
races = """
SELECT *
FROM races
"""

In [308]:
lap_times = """
SELECT *
FROM lap_times"""

In [362]:
pit_stops = """
SELECT *
FROM pit_stops
"""

In [392]:
df_joined=data_retr(joined)

In [394]:
df_joined.head()

,raceId,driverId,lt_lap,position,milliseconds,lap,circuitId,date
0,841,20,1,1,98109,1,1,2011-03-27
1,841,20,2,1,93006,2,1,2011-03-27
2,841,20,3,1,92713,3,1,2011-03-27
3,841,20,4,1,92803,4,1,2011-03-27
4,841,20,5,1,92342,5,1,2011-03-27


In [377]:
df_races = data_retr(races)
df_races.head()

,raceId,year,round,circuitId,name,date,time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00


In [389]:
df_races.shape

(1039, 7)

In [390]:
df_lap_times.shape

(472504, 6)

In [378]:
df_lap_times = data_retr(lap_times)
df_lap_times.head()

,raceId,driverid,Lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [363]:
df_pit_stops = data_retr(pit_stops)


In [324]:
df_pit_stops.columns

Index(['raceId', 'driverId', 'stop', 'lap', 'time', 'duration', 'millisec'], dtype='object')

In [ ]:
# if raceid and driverid and lap in ps is the same as raceid and driverid and lap in lt
df_pit_stops.

In [293]:
test = """
            SELECT *
            FROM pit_stops 
            where raceId = 944;
            
            
"""

In [296]:
bar = data_retr(test)

In [219]:
test = """
SELECT *
FROM lap_times AS lt
WHERE driverid IN (SELECT driverid
                FROM results AS r
                WHERE (r.position = '1') AND r.raceId != 1030)
                AND
        raceId IN (SELECT raceId
                   FROM results
                   WHERE (position = '1') AND raceId != 1030)
                AND
        driverid IN (SELECT driverid
                     FROM results
                     WHERE (position = '1') AND raceId != 1030)
ORDER BY raceId, Lap
"""

In [5]:
def data_retr(input_str):
    cur.execute(input_str)
    table = cur.fetchall()
    a = list(cur.description)
    column_names = [str(a[x][0]) for x in range(len(a))]
    return pd.DataFrame(list(table),columns=column_names)


In [220]:
try_this = data_retr(test)

In [198]:
results_table = data_retr(results)

In [64]:
a =list(desc)

In [77]:
[str(a[x][0]) for x in range(len(list(cur.description)))]

['raceId',
 'driverid',
 'Lap',
 'position',
 'time',
 'milliseconds',
 'raceId',
 'driverId',
 'stop',
 'lap',
 'time',
 'duration',
 'millisec']